In [ ]:
import pandas as pd
import numpy as np
import os
import re
import string
from collections import defaultdict
import unidecode 

In [ ]:
df = pd.read_json("../data/yelp_photos/photos.json", lines = True)

In [ ]:
df.head(20)

In [ ]:
df['caption'][0]

In [ ]:
missing_idx = df['caption'] == ''

In [ ]:
df_full = df[np.logical_not(missing_idx)]
df_missing = df[missing_idx]

In [ ]:
print("Num Images with captions:    %d"%  df_full.shape[0])
print("Num Images without captions: %d"%  df_missing.shape[0])

In [ ]:
df_full[['photo_id', 'caption']].sample(20)

In [ ]:
df_full['caption'] = df_full['caption'].apply(lambda x: re.sub(string = x, pattern = r'(\n|\r)', repl = " "))

In [ ]:
df_full['caption'].replace("\&", " and ", regex = True, inplace = True)

In [ ]:
df_full.caption.replace("[a-z\:\/\.0-9]+\.(org|com|net)", " website ", regex = True, inplace = True)

In [ ]:
df_full.caption.replace("([0-9]+ |)[0-9]+", " num ", regex = True, inplace=True)

In [ ]:
df_full.caption = df_full.caption.apply(lambda x: unidecode.unidecode(x))

In [ ]:
trans_table = str.maketrans('', '', string.punctuation + "–")
df_full['caption'] = df_full['caption'].apply(lambda x: x.translate(trans_table))

In [ ]:
df_full.caption = df_full.caption.apply(lambda x: x.lower().strip())

In [ ]:
df_full[['photo_id', 'caption']].sample(20)

In [ ]:
train_idx = np.repeat(False, df_full.shape[0])
np.random.seed(8)
train_idx[np.random.choice(a = int(len(train_idx)), size = int(.8*len(train_idx)), replace = False)] = True
valid_idx = np.logical_not(train_idx)

In [ ]:
print(np.sum(np.logical_and(train_idx, valid_idx)) == 0)
print(np.sum(np.logical_or(train_idx, valid_idx)) == df_full.shape[0])


In [ ]:
df_train = df_full[['photo_id', 'caption']][train_idx]
df_valid = df_full[['photo_id', 'caption']][valid_idx]

In [ ]:
mkdir ../data/split_lists

In [ ]:
df_train.to_csv("../data/split_lists/train_ids.csv", index = False)
df_valid.to_csv("../data/split_lists/valid_ids.csv", index = False)

In [ ]:
df_missing[['photo_id']].to_csv("../data/split_lists/playground_ids.csv", index = False)

In [ ]:
ls -l ../data/split_lists/

In [ ]:
df_valid.head(20)